In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_hub as hub

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b8a75bf6-6c4a-6c9d-5fe9-86db961f5d90)


In [3]:
!wget https://github.com/Illalla/helper_functions_for_tf/raw/main/helper_functions.py

--2022-09-21 10:34:52--  https://github.com/Illalla/helper_functions_for_tf/raw/main/helper_functions.py
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Illalla/helper_functions_for_tf/main/helper_functions.py [following]
--2022-09-21 10:34:52--  https://raw.githubusercontent.com/Illalla/helper_functions_for_tf/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8636 (8.4K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   8.43K  --.-KB/s    in 0s      

2022-09-21 10:34:52 (60.6 MB/s) - ‘helper_functions.py’ saved [8636/8636

In [4]:
from helper_functions import *

In [5]:
!wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

--2022-09-21 10:34:52--  http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip [following]
--2022-09-21 10:34:53--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip’

trainingandtestdata 100%[===================>]  77.59M  91.4MB/s    in 0.8s    

2022-09-21 10:34:53 (91.4 MB/s) - ‘trainingandtestdata.zip’ saved [81363704/81363704]



In [6]:
import zipfile

zip_ref = zipfile.ZipFile('trainingandtestdata.zip')
zip_ref.extractall()
zip_ref.close()

In [7]:
train_df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None).sample(frac=1)
test_df = pd.read_csv('/content/testdata.manual.2009.06.14.csv', encoding='latin-1', header=None).sample(frac=1)
train_df.head()

,0,1,2,3,4,5
1022663,4,1882836640,Fri May 22 07:34:07 PDT 2009,NO_QUERY,MODELCHiCK2,Fuck u fridays is here!! Sooo fuck u all on be...
69618,0,1693365784,Sun May 03 22:19:45 PDT 2009,NO_QUERY,JD2653,can't believe what just happened. Why on earth...
986746,4,1834597796,Mon May 18 03:32:26 PDT 2009,NO_QUERY,ThaRebeliouZ,"Good Morning yall, ready to start off the week..."
386060,0,2053722633,Sat Jun 06 05:30:11 PDT 2009,NO_QUERY,Emberskye,"It's early saturday morning, most of the kids ..."
1270925,4,2000474770,Mon Jun 01 22:11:50 PDT 2009,NO_QUERY,Rivrob,3 more early starts and then a proper 2 day we...


- 0 - the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
- 1 - the id of the tweet (2087)
- 2 - the date of the tweet (Sat May 16 23:58:44 UTC 2009)
- 3 - the query (lyx). If there is no query, then this value is NO_QUERY.
- 4 - the user that tweeted (robotickilldozr)
- 5 - the text of the tweet (Lyx is cool)

In [8]:
train_df = train_df[[0, 4, 5]]
test_df = test_df[[0, 4, 5]]
train_df.head()

,0,4,5
1022663,4,MODELCHiCK2,Fuck u fridays is here!! Sooo fuck u all on be...
69618,0,JD2653,can't believe what just happened. Why on earth...
986746,4,ThaRebeliouZ,"Good Morning yall, ready to start off the week..."
386060,0,Emberskye,"It's early saturday morning, most of the kids ..."
1270925,4,Rivrob,3 more early starts and then a proper 2 day we...


In [9]:
train_target = train_df[0].to_numpy()
train_usernames = train_df[4].to_numpy()
train_tweets = train_df[5].to_numpy()
test_target = test_df[0].to_numpy()
test_usernames = test_df[4].to_numpy()
test_tweets = test_df[5].to_numpy()

In [10]:
train_df[0].value_counts()

4    800000
0    800000
Name: 0, dtype: int64

In [11]:
test_df[0].value_counts()

4    182
0    177
2    139
Name: 0, dtype: int64

It seems like we cannot just predict a category. It would be better to normalise the targets. Then values closer to 0 would be negative whereas values closer to 1 would be positive.


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(train_target.reshape(-1, 1))
train_targets = tf.cast(scaler.transform(train_target.reshape(-1, 1)), dtype=tf.float32)
test_targets = tf.cast(scaler.transform(test_target.reshape(-1, 1)), dtype=tf.float32)

In [13]:
train_targets.shape

TensorShape([1600000, 1])

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor

model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('reg', SGDRegressor())
])

model_0.fit(train_tweets, train_targets)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('reg', SGDRegressor())])

In [15]:
model_0.score(test_tweets, test_targets)

0.2779722983671191

In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
def regression_scores(y_true, y_pred):
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  return {'mae': mae,
          'mse': mse}

In [17]:
y_pred = model_0.predict(test_tweets)
model_0_scores = regression_scores(test_targets, y_pred)
model_0_scores

{'mae': 0.308376582205192, 'mse': 0.13010627643053577}

In [18]:
tweet_lengths = [len(tweet.split()) for tweet in train_tweets]
np.mean(tweet_lengths), np.max(tweet_lengths)

(13.176150625, 64)

In [19]:
np.percentile(tweet_lengths, 95)

25.0

In [20]:
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                    pad_to_max_tokens=True,
                                                    output_sequence_length=25)
text_vectorizer.adapt(train_tweets[:100000])

In [21]:
vocab = text_vectorizer.get_vocabulary()

In [22]:
embedding = tf.keras.layers.Embedding(25,
                                      256,
                                      mask_zero=True,
                                      input_length=len(vocab))

In [23]:
text_vectorized = text_vectorizer(train_tweets[:100])
text_vectorized[0], text_vectorized.shape

(<tf.Tensor: shape=(25,), dtype=int64, numpy=
 array([ 503,   61, 2469,    9,   86,  417,  503,   61,   35,   15, 9508,
          13,    1, 3997,    9,  291, 1905,   25,   44,  154, 1938, 2979,
           1,  669,    0])>, TensorShape([100, 25]))

In [24]:
text_embed = embedding(text_vectorized)
text_embed[0], text_embed.shape

(<tf.Tensor: shape=(25, 256), dtype=float32, numpy=
 array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-0.00681506,  0.04780071, -0.01009294, ..., -0.04428514,
          0.0306274 ,  0.00488953],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.00108193, -0.03133496,  0.00364133, ...,  0.03812322,
         -0.02690302, -0.04891543]], dtype=float32)>,
 TensorShape([100, 25, 256]))

The train dataset is huge so I'll use only 1% of it

In [25]:
split = int(0.01*len(train_tweets))
split

16000

In [26]:
train_vectorized = tf.cast(text_vectorizer(train_tweets[:split]), dtype=tf.float32)
test_vectorized = text_vectorizer(test_tweets)

In [27]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(25,))
x = embedding(inputs)
x = layers.LSTM(128, activation='relu')(x)
outputs = layers.Dense(1)(x)

model_1 = tf.keras.Model(inputs, outputs)

In [28]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25)]              0         
                                                                 
 embedding (Embedding)       (None, 25, 256)           6400      
                                                                 
 lstm (LSTM)                 (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 203,649
Trainable params: 203,649
Non-trainable params: 0
_________________________________________________________________


In [29]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_vectorized, train_targets[:split])).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_vectorized, test_targets)).batch(32).prefetch(tf.data.AUTOTUNE)

In [30]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 25), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [31]:
model_1.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer='adam')

history_1 = model_1.fit(train_dataset, steps_per_epoch=len(train_dataset), validation_data=test_dataset,
                        validation_steps=len(test_dataset), epochs=30,
                        callbacks=[create_cp_callback('model_1'), tf.keras.callbacks.EarlyStopping(patience=5)])

Epoch 1/30
500/500 [==============================] - ETA: 0s - loss: 0.4395

500/500 [==============================] - 44s 79ms/step - loss: 0.4395 - val_loss: 0.4528
Epoch 2/30
500/500 [==============================] - 37s 74ms/step - loss: 0.4040 - val_loss: 0.4638
Epoch 3/30
500/500 [==============================] - 36s 72ms/step - loss: 0.3977 - val_loss: 0.4592
Epoch 4/30
500/500 [==============================] - 36s 73ms/step - loss: 0.3954 - val_loss: 0.4607
Epoch 5/30
500/500 [==============================] - 36s 73ms/step - loss: 0.3940 - val_loss: 0.4676
Epoch 6/30
500/500 [==============================] - 35s 71ms/step - loss: 0.3944 - val_loss: 0.4569


In [32]:
model_1 = tf.keras.models.load_model('/content/experiments/checkpoints/model_1')

In [33]:
y_preds = model_1.predict(text_vectorizer(test_tweets))
model_1_scores = regression_scores(test_targets, y_preds)
model_1_scores

{'mae': 0.45284972, 'mse': 0.31555977}

In [34]:
model_2 = tf.keras.Sequential([
    layers.Lambda(lambda x: tf.expand_dims(x, axis=1)),
    layers.Conv1D(64, 3, padding='same'),
    layers.Conv1D(64, 3, padding='same'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)
])

model_2.compile(loss='mae', optimizer='adam')

history_2 = model_2.fit(train_dataset, steps_per_epoch=len(train_dataset), validation_data=test_dataset,
                        validation_steps=len(test_dataset), epochs=60,
                        callbacks=[create_cp_callback('model_2'), tf.keras.callbacks.EarlyStopping(patience=10)],
                        verbose=0)

In [35]:
model_2 = tf.keras.models.load_model('/content/experiments/checkpoints/model_2')
y_preds = model_2.predict(text_vectorizer(test_tweets))
model_2_scores = regression_scores(test_targets, y_preds)
model_2_scores

{'mae': 0.437375, 'mse': 0.25563228}

In [36]:
encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                               input_shape=[],
                               dtype=tf.string,
                               trainable=False)

In [37]:
train_vectorized = tf.cast(encoder_layer(train_tweets[:split]), dtype=tf.float32)
test_vectorized = encoder_layer(test_tweets)
train_dataset = tf.data.Dataset.from_tensor_slices((train_vectorized, train_targets[:split])).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_vectorized, test_targets)).batch(32).prefetch(tf.data.AUTOTUNE)

In [38]:
model_3 = tf.keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(1)
])

model_3.compile(loss='mae', optimizer='adam')

history_3 = model_3.fit(train_dataset, steps_per_epoch=len(train_dataset), validation_data=test_dataset,
                        validation_steps=len(test_dataset), epochs=60,
                        callbacks=[create_cp_callback('model_3'), tf.keras.callbacks.EarlyStopping(patience=10)],
                        verbose=0)

In [39]:
model_3 = tf.keras.models.load_model('/content/experiments/checkpoints/model_3')
y_preds = model_3.predict(encoder_layer(test_tweets))
model_3_scores = regression_scores(test_targets, y_preds)
model_3_scores

{'mae': 0.26248482, 'mse': 0.12363383}

Usernames usually aren't "normal" words so it'd be better to focus of characters

In [40]:
def splitchar(text):
  return ' '.join(list(text))

In [41]:
train_usernames_chars = [splitchar(uname) for uname in train_usernames]
test_usernames_chars = [splitchar(uname) for uname in test_usernames]

In [42]:
char_usernames_lengths = [len(sentence) for sentence in train_usernames_chars]
np.mean(char_usernames_lengths), np.percentile(char_usernames_lengths, 95)

(19.32454125, 29.0)

In [43]:
char_vectorizer = tf.keras.layers.TextVectorization(max_tokens=200,
                                                    pad_to_max_tokens=True,
                                                    output_sequence_length=30)
char_vectorizer.adapt(train_usernames_chars[:10000])
char_vocab = char_vectorizer.get_vocabulary()

In [44]:
char_embedding = layers.Embedding(input_dim=30,
                                  output_dim=256)

In [45]:
train_usernames_vectorized = char_vectorizer(train_usernames_chars[:split])
test_usernames_vectorized = char_vectorizer(test_usernames_chars)

In [46]:
train_usernames_vectorized.shape, train_vectorized.shape, train_targets[:split].shape, test_usernames_vectorized.shape, test_vectorized.shape, test_targets.shape

(TensorShape([16000, 30]),
 TensorShape([16000, 512]),
 TensorShape([16000, 1]),
 TensorShape([498, 30]),
 TensorShape([498, 512]),
 TensorShape([498, 1]))

In [47]:
train_dataset_chars = tf.data.Dataset.from_tensor_slices(((train_vectorized, train_usernames_vectorized), train_targets[:split])).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset_chars = tf.data.Dataset.from_tensor_slices(((test_vectorized, test_usernames_vectorized), test_targets)).batch(32).prefetch(tf.data.AUTOTUNE)

In [48]:
train_dataset_chars

<PrefetchDataset element_spec=((TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), TensorSpec(shape=(None, 30), dtype=tf.int64, name=None)), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [49]:
text_inputs = layers.Input(shape=(512,))
x = layers.Dense(256, activation='relu')(text_inputs)
model_text = tf.keras.Model(text_inputs, x)

char_inputs = layers.Input(shape=(30,))
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(char_inputs)
x = layers.Conv1D(64, 4, padding='same')(x)
x = layers.Conv1D(64, 4, padding='same')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128)(x)
model_char = tf.keras.Model(char_inputs, x)

concat_inputs = layers.Concatenate()((model_text.output, model_char.output))
x = layers.Dense(128, activation='relu')(concat_inputs)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1)(x)

model_4 = tf.keras.Model((model_text.input, model_char.input), outputs)

model_4.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 lambda_1 (Lambda)              (None, 1, 30)        0           ['input_3[0][0]']                
                                                                                                  
 conv1d_2 (Conv1D)              (None, 1, 64)        7744        ['lambda_1[0][0]']               
                                                                                                  
 conv1d_3 (Conv1D)              (None, 1, 64)        16448       ['conv1d_2[0][0]']               
                                                                                            

In [50]:
model_4.compile(loss='mae', optimizer='adam')
history_4 = model_4.fit(train_dataset_chars, epochs=60, steps_per_epoch=len(train_dataset_chars),
                        validation_data=test_dataset_chars, validation_steps=len(test_dataset_chars),
                        callbacks=[create_cp_callback('model_4'), tf.keras.callbacks.EarlyStopping(patience=5)],
                        verbose=0)

In [51]:
model_4 = tf.keras.models.load_model('/content/experiments/checkpoints/model_4')
y_preds = model_4.predict(test_dataset_chars)
model_4_scores = regression_scores(test_targets, y_preds)
model_4_scores

{'mae': 0.2510887, 'mse': 0.1237444}

This seems to be the best result so far. It's time for the great model with 10% of the data (100% would train for too long).

In [52]:
split = int(0.1*len(train_tweets))
train_vectorized = tf.cast(encoder_layer(train_tweets[:split]), dtype=tf.float32)
train_usernames_vectorized = char_vectorizer(train_usernames_chars[:split])
train_dataset_chars = tf.data.Dataset.from_tensor_slices(((train_vectorized, train_usernames_vectorized), train_targets[:split])).batch(32).prefetch(tf.data.AUTOTUNE)

In [53]:
model_5 = tf.keras.models.clone_model(model_4)

In [54]:
model_5.compile(loss='mae', optimizer='adam')
history_5 = model_5.fit(train_dataset_chars, epochs=5, steps_per_epoch=len(train_dataset_chars),
                        validation_data=test_dataset_chars, validation_steps=len(test_dataset_chars),
                        callbacks=[create_cp_callback('model_5')],
                        verbose=1)

Epoch 1/5
5000/5000 [==============================] - 21s 4ms/step - loss: 0.2531 - val_loss: 0.2636
Epoch 2/5
5000/5000 [==============================] - 19s 4ms/step - loss: 0.2223 - val_loss: 0.2713
Epoch 3/5
5000/5000 [==============================] - 19s 4ms/step - loss: 0.2132 - val_loss: 0.2654
Epoch 4/5
5000/5000 [==============================] - 19s 4ms/step - loss: 0.2066 - val_loss: 0.2650
Epoch 5/5
5000/5000 [==============================] - 19s 4ms/step - loss: 0.2000 - val_loss: 0.2733


Main problem with this dataset is that in training we have only 2 categories whereas in test data there are 3 categories. Thus it's difficult not to overfit the model.